There are two hand-in exercises for this topic: 
i. 
The first one is explained in bullet 2 of slide 16 of the slide deck “IoT & Sensor 
Data.pptx”. You need to do the entire task described in bullet 2 of slide 16. 
ii. 
The second one is related to PCA and it is explained in the notebook “pdm_task”. 
Note that the data simulation parts are already done for you, so you do not need to 
do those parts. You should start from the cell where tasks are listed and you n*eed 
to do all 5 tasks. 

Use the diabetes.csv dataset to do the following:
1. Select the following 4 attributes (3 features + 1 class label) :
   - Glucose, BloodPressure, Insulin, Outcome
3. Normalize Glucose, BloodPressure and Insulin to [0, 1] using MinMax.
4. Store the new data (3 normalized features + 1 class label) in another dataset S.
5. Modify the MQTT example to do the following:
   - The publisher publishes records in S continuously. When it reaches the end of S, it continues to send from the beginning again.
   - The subscriber continuously receives the data. For each latest record r received, apply the 3NN classification to the last 5 records before r, and compare the classification result with the Outcome label in r.
   - Repeat this for 1000 times, and report the number of correct classifications. 


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time
import paho.mqtt.client as mqtt


In [2]:
df = pd.read_csv("diabetes.csv")
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
# Select the required columns
selected_columns = ["Glucose", "BloodPressure", "Insulin", "Outcome"]
df_selected = df[selected_columns]

In [4]:
df_selected

,Glucose,BloodPressure,Insulin,Outcome
0,148,72,0,1
1,85,66,0,0
2,183,64,0,1
3,89,66,94,0
4,137,40,168,1
...,...,...,...,...
763,101,76,180,0
764,122,70,0,0
765,121,72,112,0
766,126,60,0,1


In [5]:
# Select the columns to normalize
GBI_normalize = ['Glucose', 'BloodPressure', 'Insulin']

scaler = MinMaxScaler()

# Fit and transform the selected columns
df_selected.loc[:, GBI_normalize] = scaler.fit_transform(df_selected[GBI_normalize])

C:\Users\cbech\AppData\Local\Temp\ipykernel_22524\2730066625.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.74371859 0.42713568 0.91959799 0.44723618 0.68844221 0.58291457
 0.3919598  0.57788945 0.98994975 0.6281407  0.55276382 0.84422111
 0.69849246 0.94974874 0.83417085 0.50251256 0.59296482 0.53768844
 0.51758794 0.57788945 0.63316583 0.49748744 0.98492462 0.59798995
 0.71859296 0.6281407  0.73869347 0.48743719 0.72864322 0.5879397
 0.54773869 0.79396985 0.44221106 0.46231156 0.61306533 0.51758794
 0.69346734 0.51256281 0.45226131 0.55778894 0.90452261 0.66834171
 0.53266332 0.85929648 0.79899497 0.90452261 0.73366834 0.35678392
 0.51758794 0.52763819 0.51758794 0.50753769 0.44221106 0.88442211
 0.75376884 0.36683417 0.93969849 0.50251256 0.73366834 0.52763819
 0.42211055 0.66834171 0.22110553 0.70854271 0.57286432 0.49748744
 0.54773869 0.54773869 0.47738693 0.73366834 0.50251256 0.69849246
 0.63316583 

In [6]:
df_selected.head()


,Glucose,BloodPressure,Insulin,Outcome
0,0.743719,0.590164,0.000000,1
1,0.427136,0.540984,0.000000,0
2,0.919598,0.524590,0.000000,1
3,0.447236,0.540984,0.111111,0
4,0.688442,0.327869,0.198582,1


In [7]:
S = df_selected.copy()

In [ ]:
import time
import paho.mqtt.client as mqtt

# We create a client as the data publisher
mqttc = mqtt.Client()
# We are connecting to port 1883 and set *keepalive* parameter to 60; *keepalive* is the maximum waiting time in seconds.
mqttc.connect("mqtt.eclipseprojects.io", 1883, 60)

# We send the following readings repeatedly. Once we reach the end, we start from the beginning again.
data = [S["Glucose"].values, S["BloodPressure"].values, S["Insulin"].values]
period = len(data)

# We send the following readings repeatedly. Once we reach the end, we start from the beginning again.
data = S.to_dict(orient='records')
period = len(data)


if __name__ == "__main__":
    print("Publishing...")
    
    index = 0
    while (index >= 0):
        # Get the current data reading to send out
        record = data[index % period]
        
        # Publish the data reading as 'myroom/temperature'
        #Publisher is open to all! Make sure use your own ‘Data name’ in your exercise of Publisher/Subscriber
        mqttc.publish("Records", str(record))
        
        # We send the next reading after 2 seconds
        time.sleep(0.5)
        index = index + 1

Publishing...
